In [1]:
import os

import numpy as np

from sklearn.preprocessing import OneHotEncoder

# Load data from each folder in /data/toPreProc/

data_in = 0 # Number of input dimensions to the model
data_out = 0 # Number of output dimensions to the model

data_dict = {}
label_dict = {}

for folder in os.listdir('data/toPreProc/'):
    print("Folder name: ", folder)
    data_dict[folder] = np.load(f'data/toPreProc/{folder}/{folder}PreprocessedData.npy')
    labels = np.load(f'data/toPreProc/{folder}/{folder}Labels.npy', allow_pickle=True)
    label_dict[folder] = labels[:-1, 2]
    print("Data shape: ", data_dict[folder].shape)
    print("Label shape: ", label_dict[folder].shape)
    if(data_dict[folder].shape[1] > data_in):
        data_in = data_dict[folder].shape[1]
    if(label_dict[folder].shape[0] > data_out):
        data_out = label_dict[folder].shape[0]

# # One hot encode the labels
# ohes = {}
# num_of_labels = len(label_dict) # Includes one extra for the end of the sequence
# for key, y in label_dict.items():
#     #

#     #
#     print(y.reshape(-1, 1))
#     ohes[key] = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit(y)
#     label_dict[key] = ohes[key].transform(y)
#     print(label_dict[key])

Folder name:  2023-12-18_003
Data shape:  (26, 100, 68)
Label shape:  (26,)
Folder name:  2023-12-15_002
Data shape:  (26, 100, 68)
Label shape:  (26,)
Folder name:  2023-12-17_002
Data shape:  (26, 100, 68)
Label shape:  (26,)
Folder name:  2023-12-17_001
Data shape:  (26, 100, 68)
Label shape:  (26,)
Folder name:  2023-12-18_002
Data shape:  (26, 100, 68)
Label shape:  (26,)
Folder name:  2023-12-18_001
Data shape:  (26, 100, 68)
Label shape:  (26,)


In [8]:
import torch
import random

import torch.nn as nn

dropout_prob = 0.2

# Define the sequential model
model = nn.Sequential(
    nn.BatchNorm1d(data_in),
    nn.Dropout1d(dropout_prob),
    nn.Conv1d(kernel_size=5, padding_mode="replicate", in_channels=data_in, out_channels=64),
    nn.MaxPool1d(2),
    nn.Flatten(),
    nn.Dropout1d(dropout_prob),
    nn.Conv1d(kernel_size=3, padding_mode="replicate", in_channels=64, out_channels=64),
    nn.MaxPool1d(2),
    nn.Flatten(),
    nn.Dropout1d(dropout_prob),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Dropout1d(dropout_prob),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Dropout1d(dropout_prob), 
    nn.Linear(32, data_out),
    nn.Softmax(dim=1)
)
compiled_model = torch.compile(model)
#model.compile(optimizer='adam', loss=nn.CrossEntropyLoss, metrics=['accuracy'])

In [50]:
batch_size = 2

epochs = 5

num_batches = (data_out // batch_size) # Number of batches to run based on the batch size and the number of samples

loss_fn = nn.CrossEntropyLoss

optimizer = torch.optim.Adam(compiled_model.parameters(), lr=0.001)

batches = []
labels = []

for value in list(data_dict.values()):
    #print(value)
    batch = []
    batch = value.reshape(13, 2, 100, 68)
    batch = np.array(batch)
    batches.append(batch)
print(np.shape(batches))

for i in range(num_batches):
    for value in list(label_dict.values())[i*batch_size:(i+1)*batch_size]:
        labels.append(value)

labels_in_batches = np.array(labels)
print(np.shape(labels_in_batches))
labels_in_batches = labels_in_batches.reshape(6, -1, 2)
print(np.shape(labels_in_batches))

data_in_batches = np.array(batches)

print(np.shape(data_in_batches))

(6, 13, 2, 100, 68)
(6, 26)
(6, 13, 2)
(6, 13, 2, 100, 68)


In [57]:


for index, trial in enumerate(data_in_batches):
    
    optimizer.zero_grad() # Reset the gradients

    print(np.shape(trial))
    for index2, batch in enumerate(trial):
        print(np.shape(batch))
        batch = torch.from_numpy(batch)
        print(np.shape(batch))
        out = compiled_model(batch)
        
        loss = loss_fn(out, labels_in_batches[index2])
        loss.backward()
        optimizer.step()

(13, 2, 100, 68)
(2, 100, 68)
torch.Size([2, 100, 68])


TorchRuntimeError: Failed running call_module self_6(*(FakeTensor(..., size=(s0, 2048), dtype=torch.float64,
           grad_fn=<SqueezeBackward1>),), **{}):
Invalid channel dimensions

from user code:
   File "/home/ebuntel/.local/lib/python3.10/site-packages/torch/nn/modules/container.py", line 215, in forward
    input = module(input)

Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information


You can suppress this exception and fall back to eager by setting:
    import torch._dynamo
    torch._dynamo.config.suppress_errors = True
